In [22]:
import pandas as pd
import numpy as np
from time import time
from collections import Counter
import collections

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk

In [1]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [2]:
def getInterest(_list, arr_index):
    return _list[arr_index]

df_train['POI'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(0,))
df_train['street'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(1,))
df_train.head(20)

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",/kem mel raya,,kem mel raya
7,7,tela keuramat kuta alam,/tela,,tela
8,8,gg. i wates magersari,/gg. i,,gg. i
9,9,bunga ncole ix 2,/bunga ncole ix,,bunga ncole ix


In [3]:
def getListOfWords(sentence):
    sentence = sentence.replace('\n', ' ').replace('\t', ' ')
    sentence = sentence.replace('/', ' ').replace('.', ' ').replace(',', ' ')
    words = sentence.split(' ')
    while '' in words:
        words.remove('')
        
    return [word for word in words if len(words) > 0]

In [4]:
def getDifference(row):
    raw = getListOfWords(row['raw_address'])
    poi_street = getListOfWords(row['POI/street'])
    temp = set(raw)
    output = [x for x in poi_street if x not in temp]
    if len(output) > 0:
        return output
    else:
        return np.nan

In [5]:
start = time()

df_train['delta_words'] = df_train.apply(getDifference, axis=1)

print("Executed in {} seconds.".format(time() - start))

Executed in 5.626911401748657 seconds.


In [6]:
df_need_repair = df_train.dropna()
df_need_repair

,id,raw_address,POI/street,POI,street,delta_words
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]"
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]"
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,,[bangunan]
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]"
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri]
...,...,...,...,...,...,...
299973,299973,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad]
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center]
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket]
299986,299986,"ginzi cafe, siliw,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi]


# N-gram analysis

## find the pair

In [20]:
def getDistance(str1, str2):
    return nltk.edit_distance(str1, str2)

getDistance('negeri', 'neg')

3

In [14]:
for ngram in nltk.ngrams(getListOfWords("cikahuripan sd neg boj 02 klap boj, no 5 16877"), 2):
    print (' '.join(ngram))

cikahuripan sd
sd neg
neg boj
boj 02
02 klap
klap boj
boj no
no 5
5 16877


In [30]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(x.items(), key=lambda kv: kv[1])
sorted_dict = collections.OrderedDict(sorted_x)
sorted_x[1][0]

2

In [39]:
def getPair(row):
    raw = row['raw_address']
    repaired = row['delta_words']
    _dict1 = {}
    _dict2 = {}
    for repaired_word in repaired:
        for raw_word in getListOfWords(raw):
            _dict2[raw_word] = getDistance(repaired_word, raw_word)
        # get minimum distance word
        sorted_dict2 = sorted(_dict2.items(), key=lambda kv: kv[1])
        _dict1[repaired_word] = sorted_dict2[0][0]
        # inverse dict from raw to repaired
        inv_map = {v: k for k, v in _dict1.items()}
    return inv_map

In [41]:
df_need_repair['Pair'] = df_need_repair.apply(getPair, axis=1)
df_need_repair

<ipython-input-41-36cfc8fdbb57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_repair['Pair'] = df_need_repair.apply(getPair, axis=1)


,id,raw_address,POI/street,POI,street,delta_words,Pair
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]","{'neg': 'negeri', 'boj': 'bojong'}"
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]","{'yaya': 'yayasan', 'atohar': 'atohariyah'}"
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,,[bangunan],{'bang': 'bangunan'}
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]","{'mar': 'markaz', 'tabl': 'tabligh'}"
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri],{'neg': 'negeri'}
...,...,...,...,...,...,...,...
299973,299973,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad],{'moha': 'mohammad'}
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center],{'cen': 'center'}
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket],{'minima': 'minimarket'}
299986,299986,"ginzi cafe, siliw,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi],{'siliw': 'siliwangi'}


In [59]:
corpus = []
for index, row in df_need_repair.iterrows():
    for key, value in row['Pair'].items():
        temp = [key,value]
        corpus.append(temp)
corpus = Counter(tuple(item) for item in corpus)
corpus = dict(sorted(corpus.items(), key=lambda item: item[1], reverse=True))

In [55]:
x = {}
x['a'] = [{'abc': 190}]
x['a'].append({'abd': 100})

{'a': [{'abc': 190}, {'abd': 100}]}

In [57]:
if 'b' in x:
    x['b'].append({'bcd': 300})
else:
    x['b'] = [{'bcd': 300}]
x

{'a': [{'abc': 190}, {'abd': 100}], 'b': [{'bcd': 300}]}

In [60]:
dict_1gram = {}
for key, value in corpus.items():
    if key[0] in dict_1gram:
        dict_1gram[key[0]].append({key[1]: value})
    else:
        dict_1gram[key[0]] = [{key[1]: value}]
dict_1gram

{'war': [{'warung': 1478},
  {'warkop': 218},
  {'wartel': 82},
  {'warteg': 67},
  {'warnet': 54},
  {'warnas': 6},
  {'warunk': 6},
  {'warkom': 5},
  {'wardah': 4},
  {'waronk': 2},
  {'warudo': 2},
  {'warsih': 2},
  {'warnes': 1},
  {'damen': 1},
  {'wasrun': 1},
  {'waries': 1},
  {'warnis': 1},
  {'warsab': 1},
  {'wareeg': 1},
  {'wariah': 1},
  {'warsan': 1},
  {'warles': 1},
  {'wareng': 1},
  {'wartek': 1},
  {'warter': 1},
  {'warkan': 1},
  {'warong': 1},
  {'warlap': 1},
  {'wartaw': 1},
  {'rapa': 1},
  {'warsun': 1},
  {'warbon': 1}],
 'neg': [{'negeri': 1056}, {'negara': 107}],
 'mas': [{'masjid': 1054},
  {'master': 26},
  {'masehi': 6},
  {'masnur': 2},
  {"mas'ud": 2},
  {'masayu': 1},
  {'masdah': 1},
  {'maskur': 1},
  {'masuya': 1},
  {'massip': 1},
  {'massie': 1},
  {'masoem': 1},
  {'mastun': 1},
  {'muyass': 1},
  {'mastur': 1},
  {'maskan': 1},
  {'mastam': 1},
  {'masken': 1},
  {'masugi': 1},
  {'masril': 1},
  {'maskos': 1},
  {'maslah': 1},
  {'mashur': 

## To pandas series

In [9]:
def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words]

In [12]:
words_raw = basic_clean(''.join(str(df_need_repair['raw_address'].tolist())))
words_poi = basic_clean(''.join(str(df_train['POI/street'].tolist())))

In [28]:
raw_1gram = pd.Series(nltk.ngrams(words_raw, 1)).value_counts()
raw_2gram = pd.Series(nltk.ngrams(words_raw, 2)).value_counts()
raw_3gram = pd.Series(nltk.ngrams(words_raw, 3)).value_counts()
poi_2gram = pd.Series(nltk.ngrams(words_poi, 2)).value_counts()
poi_3gram = pd.Series(nltk.ngrams(words_poi, 3)).value_counts()

In [45]:
raw_1gram.loc('rt')

ValueError: No axis named rt for object type Series

<generator object ngrams at 0x0000024394E28040>